# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Initialize-Environment" data-toc-modified-id="Initialize-Environment-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Initialize Environment</a></div><div class="lev2 toc-item"><a href="#Generate-List-of-Data" data-toc-modified-id="Generate-List-of-Data-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Generate List of Data</a></div><div class="lev1 toc-item"><a href="#Subgraph-Topography" data-toc-modified-id="Subgraph-Topography-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Subgraph Topography</a></div><div class="lev2 toc-item"><a href="#Topography-Map" data-toc-modified-id="Topography-Map-21"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Topography Map</a></div><div class="lev2 toc-item"><a href="#Relate-Subgraph-Connection-Strength-to-Connection-Distance" data-toc-modified-id="Relate-Subgraph-Connection-Strength-to-Connection-Distance-22"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Relate Subgraph Connection Strength to Connection Distance</a></div><div class="lev2 toc-item"><a href="#Plot-Ranked-Topographical-Sensitivity" data-toc-modified-id="Plot-Ranked-Topographical-Sensitivity-23"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Plot Ranked Topographical Sensitivity</a></div>

# Initialize Environment

In [ ]:
try:
    %load_ext autoreload
    %autoreload 2
    %reset
except:
    print 'NOT IPYTHON'

from __future__ import division

import os
import sys
import glob

import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import scipy.io as io
import h5py
import matplotlib.pyplot as plt
from matplotlib import rcParams

sys.path.append('/Users/akhambhati/Developer/hoth_research/Echobase')
import Echobase
convert_conn_vec_to_adj_matr = Echobase.Network.Transforms.configuration.convert_conn_vec_to_adj_matr
convert_adj_matr_to_cfg_matr = Echobase.Network.Transforms.configuration.convert_adj_matr_to_cfg_matr

rcParams = Echobase.Plotting.fig_format.update_rcparams(rcParams)

path_CoreData = '/Users/akhambhati/Remotes/CORE.fMRI_multiband.mmattar/restdata'
path_PeriphData = '/Users/akhambhati/Remotes/RSRCH.NMF_Subnetworks'
path_SubData = path_PeriphData + '/e03a-DynFuncSubgraph-Subject'
path_ExpData = path_PeriphData + '/e05a-CmpTopographySubgraph-Subject'

for path in [path_CoreData, path_PeriphData, path_SubData, path_ExpData]:
    if not os.path.exists(path):
        print('Path: {}, does not exist'.format(path))
        os.makedirs(path)     
        
if not os.path.exists('./e05a-Figures'):
    os.makedirs('./e05a-Figures')

## Generate List of Data

In [ ]:
subj_cfg_ids = np.unique([full_subj_path.split('/')[-1].split('.')[0]
                          for full_subj_path in glob.iglob('{}/*.cfg_matr.npz'.format(path_SubData))])
    
subj_dict = {}
for subj in subj_cfg_ids:
    subj_dict[subj] = {'subgraph_path': '{}/{}.consensus_subgraph.npz'.format(path_SubData, subj),
                       'cfg_path': '{}/{}.cfg_matr.npz'.format(path_SubData, subj)}

# Subgraph Topography

## Relate Subgraph Connection Strength to Connection Distance

In [ ]:
dist_matr = np.load('{}/geom_adj_matr.npz'.format(path_SubData))['dist_matr']

%matplotlib inline
n_null = 10

n_subj = len(subj_dict.keys())
for subj_i, subj in enumerate(subj_dict.iterkeys()):
    print(" -- Processing: {}".format(subj))
    
    data_sub = np.load(subj_dict[subj]['subgraph_path'], mmap_mode='r')
    
    fac_subgraph = data_sub['fac_subnet']
    fac_coef = data_sub['fac_coef']
        
    n_fac = fac_subgraph.shape[0]
    n_conn = fac_subgraph.shape[1]
    
    conn_dist = convert_adj_matr_to_cfg_matr(dist_matr.reshape(1, dist_matr.shape[0], dist_matr.shape[1])).reshape(-1)
    
    if subj_i == 0:
        topo_sens_pop = np.nan*np.zeros((n_subj, n_fac))
        topo_sens_pop_geom_null = np.nan*np.zeros((n_subj, n_null, n_fac))

    # Correlation between Subgraph and Distance Matrix
    for f_id in xrange(n_fac):
        fac_subgraph_norm = fac_subgraph[f_id, :] / fac_subgraph[f_id, :].max()
        rho, pval = stats.pearsonr(conn_dist, fac_subgraph_norm)
        topo_sens_pop[subj_i, f_id] = rho
    
    # Null Correlation between Subgraph and Distance Matrix
    for ns in xrange(n_null):
        for f_id in xrange(n_fac):
            adj_real = convert_conn_vec_to_adj_matr(fac_subgraph[f_id, :])
            adj_null = Echobase.Network.Rewire.geometry.bin_perm(adj_real, dist_matr, 3)
            fac_subgraph_null = convert_adj_matr_to_cfg_matr(adj_null.reshape(1, adj_null.shape[0], adj_null.shape[1])).reshape(-1)
            fac_subgraph_null /= fac_subgraph_null.max()
            rho, pval = stats.pearsonr(conn_dist, fac_subgraph_null)
            topo_sens_pop_geom_null[subj_i, ns, f_id] = rho            
            
np.savez('{}/Topographic_Sensitivity.npz'.format(path_ExpData),
         topo_sens_pop=topo_sens_pop,
         topo_sens_pop_geom_null=topo_sens_pop_geom_null)

## Plot Example Geometric Null
plt.figure()
ax = plt.subplot(121)
ax.matshow(adj_real, cmap='rainbow')
ax.set_title('Real Adjacency')
ax.set_axis_off()

ax = plt.subplot(122)
ax.matshow(adj_null, cmap='rainbow')
ax.set_title('Geometric Null Adjacency')
ax.set_axis_off()

## Plot Ranked Topographical Sensitivity

In [ ]:
df = np.load('{}/Topographic_Sensitivity.npz'.format(path_ExpData))
topo_sens_pop = df['topo_sens_pop']
topo_sens_pop_geom_null = df['topo_sens_pop_geom_null']

# Sort the topographic sensitivity for subgraphs of each subject
topo_sens_pop_sorted = np.sort(topo_sens_pop, axis=1)
topo_sens_pop_geom_null_sorted = np.sort(topo_sens_pop_geom_null, axis=2).reshape(n_subj*n_null, n_fac)

alpha = 0.05 / n_fac

print('\n\n----- Geometric Null ------')
for ii in xrange(n_fac):
    tval, pval = stats.ttest_ind(topo_sens_pop_sorted[:, ii], topo_sens_pop_geom_null_sorted[:, ii])
    if pval < alpha:
        sig = '*'
    else:
        sig = ' '    
    print('{}- t={:0.5}, p={:0.5}, sig={}'.format(ii+1, tval, pval, sig))

    
# Plot Topographical Sensitivity
plt.figure()
ax = plt.subplot(111)

ax.plot(np.arange(1, n_fac+1),
        np.percentile(topo_sens_pop_geom_null_sorted, 50, axis=0),
        color='k', linewidth=1.0)
ax.fill_between(np.arange(1, 17),
                y1=np.percentile(topo_sens_pop_geom_null_sorted, 25, axis=0),
                y2=np.percentile(topo_sens_pop_geom_null_sorted, 75, axis=0),
                color=[0.75, 0.75, 0.75])

clr_list = []
for ii in xrange(n_fac):
    clr_list.append([0.0, 0.375, 1.0])
bplot = ax.boxplot(topo_sens_pop_sorted, patch_artist=True);
Echobase.Plotting.fig_format.set_box_color(bplot, 'k', clr_list)

# Axis Settings
ax.set_xlim([0, 17])
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
ax.set_ylabel('Pearson Correlation\n(Edge Strength vs Euclidean Dist.)');
ax.set_xlabel('Subgraphs');

plt.savefig('./e05a-Figures/Topographic_Sensitivity-SubgraphPCorr.svg')
plt.show()
plt.close()              